In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import math
import time
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#function that will create these 1m samples, n= smaple count 
def sample_generator(n):
    x0 = np.ones(n).reshape(n,1) #intercept value 
    #Sample X1 ~ N(3,4) and sample X2 ~ N(-1, 4)
    x1 = np.random.normal(3,4,n).reshape(n,1) 
    x2 = np.random.normal(-1,4,n).reshape(n,1)
    #theta is fixed with values 3, 1, 2 for generating the data
    theta = np.array([3,1,2]).reshape(3,1)
    #Sample epsilon error in the data from N(0, 2)
    episilon =  np.random.normal(0,math.sqrt(2),n).reshape(n,1) 
    #Make a 1Mx3 matrix
    X = np.hstack((x0, x1, x2)) #feature matrix
    h0 = np.dot(X,theta) #hypotheis function 
    #Generate the value of Y (given X, parameterized by given Theta)
    y = h0 + episilon 
    return X,y

In [ ]:
# the function to calculate the loss 
def l_theta(X, Y, theta):
    m = len(X)
    h0 = np.dot(X, theta)
    loss = h0 - Y
    return loss

In [ ]:
#using direct formula to impute the cost
def j_theta(X, Y,theta):
    m = len(X)
    loss = l_theta(X, Y, theta)
    cost = np.sum(loss **2)
    cost = cost/(2*m)
    return cost

In [ ]:
# r= number of batches 
#b= batch size 
def batch_generator(X,Y,b,r):
    m = len(X)
    left_index = b*r  #is the starting point of the batch 
    right_index = left_index + r #end of the batch 
    if m < right_index:
        new_index = left_index+(right_index - m) #until m will be less than the batch batch end size, then the new index will take the place of left index 
        Xslice = X[left_index:new_index]   
        Yslice = Y[left_index:new_index]
    else:
        Xslice = X[left_index:right_index]  #if the right index> m, then it will keep updating it, the left index will have the value of right index 
        Yslice = Y[left_index:right_index]
    return (Xslice,Yslice)

In [ ]:
def SGD(X,Y,m,r):
    #will initliaze these parameters to zero 
    theta=np.array([0,0,0]).reshape(3,1)
    epoch = 0
    not_converged = True #intially not converged 
    old_cost = 100
    iteration=0
    total_iterations = 0
    #all these parameters will change over each iteration, so I have intialized them as empty arrays that will be updated after evry iteration. 
    cost_arr = []
    theta0 = []
    theta1 = []
    theta2 = []
    episilon = 1e-6 #10^-6
    
    while(not_converged):

        print('epoch :', epoch)
        print('Updated theta:',theta)
        epoch +=1
        #m/r= b gives us the total number of batches 
        # 0.001 = learning rate 
        for b in range(0, math.ceil(m/r)):
            Xslice, Yslice = batch_generator(X,Y,b,r)
            t1 = np.dot(Xslice,theta)
            t2 = t1 - Yslice
            t3 = np.dot(Xslice.T,t2)
            t4 = 0.001*t3
            t5 = t4/r
            theta = theta - t5
            # theta = theta - 0.001*(np.dot(X_Slice.T,loss)/r) direct formula to compute theta 
            #will append all these updated values in these empty arrays 
            theta0.append(theta[0])
            theta1.append(theta[1])
            theta2.append(theta[2])
            
            present_cost = j_theta(Xslice,Yslice,theta)
            cost_arr.append(present_cost)
            iteration +=1
            total_iterations+=1
            #for  batchsize-=1 , when iterations reach 50000, convergence point is found 
            if(r == 1 and total_iterations==50000):
                not_converged = False
                break
            #for  batchesize= 100, when iterations reach 100000, the convergence point is found 
            if(r == 100 and total_iterations==100000):
                not_converged = False
                break
            #at every 1000th iteration the index restarts from zero. at every 1000th iteration the value of theta and cost are updated too 
            if(iteration == 1000):
                
                iteration=0
                list_sum = sum(cost_arr)
                new_cost =list_sum/1000
                cost_arr = []
                print('Previous Cost :',old_cost,'Next Cost :', new_cost,'Difference:', old_cost - new_cost)
                #if the difference between old cost and new cost is less thatn 10^-6 then the convergance point will be found break condition 
                if abs(old_cost - new_cost) < episilon:
                    not_converged = False
                prev_cost = new_cost
                
            
    print('No. of samples :',m,'Batch Size :',r,'Total Iterations:',total_iterations,'No of Epochs :',epoch)
    print('Final Theta :',theta)
    data = np.hstack((theta0, theta1,theta2))
    print(theta.shape)
    return data.T,theta

In [ ]:
#calculate the time taken to reach the convergence point 
#r=1
start_time = time.time()
X1,Y1 = sample_generator(1000000)
data,t2 = SGD(X1,Y1,1000000,1)
end_time = time.time()
print('Execution Time in seconds :',round(end_time - start_time,2) ) 

epoch : 0
Updated theta: [[0]
 [0]
 [0]]
Previous Cost : 100 Next Cost : 3.2261728352454977 Difference: 96.77382716475451
Previous Cost : 100 Next Cost : 1.4281772777080481 Difference: 98.57182272229196
Previous Cost : 100 Next Cost : 1.3131130356989547 Difference: 98.68688696430104
Previous Cost : 100 Next Cost : 1.0234159240298746 Difference: 98.97658407597012
Previous Cost : 100 Next Cost : 0.994183587418566 Difference: 99.00581641258144
Previous Cost : 100 Next Cost : 1.0290655096883374 Difference: 98.97093449031166
Previous Cost : 100 Next Cost : 0.9903057577995541 Difference: 99.00969424220044
Previous Cost : 100 Next Cost : 1.0026626914621213 Difference: 98.99733730853788
Previous Cost : 100 Next Cost : 1.0388166568183144 Difference: 98.96118334318169
Previous Cost : 100 Next Cost : 1.014174630758253 Difference: 98.98582536924175
Previous Cost : 100 Next Cost : 0.9479180696864203 Difference: 99.05208193031358
Previous Cost : 100 Next Cost : 0.9857502305137764 Difference: 99.0142

In [ ]:
#calculate the time taken to reach the convergence point 
#r= 100
start_time = time.time()
X1,Y1 = sample_generator(1000000)
data,t2 = SGD(X1,Y1,1000000,100)
end_time = time.time()
print('Execution Time in seconds :',round(end_time - start_time,2) )

epoch : 0
Updated theta: [[0]
 [0]
 [0]]
Previous Cost : 100 Next Cost : 3.3017610258800887 Difference: 96.69823897411992
Previous Cost : 100 Next Cost : 1.4961483106355917 Difference: 98.50385168936441
Previous Cost : 100 Next Cost : 1.2875731126348304 Difference: 98.71242688736517
Previous Cost : 100 Next Cost : 1.168381195890348 Difference: 98.83161880410965
Previous Cost : 100 Next Cost : 1.0966203499180935 Difference: 98.9033796500819
Previous Cost : 100 Next Cost : 1.0543279066495843 Difference: 98.94567209335041
Previous Cost : 100 Next Cost : 1.03129758903517 Difference: 98.96870241096482
Previous Cost : 100 Next Cost : 1.0140915873887042 Difference: 98.9859084126113
Previous Cost : 100 Next Cost : 1.020429197818272 Difference: 98.97957080218173
Previous Cost : 100 Next Cost : 1.0062407889029268 Difference: 98.99375921109707
epoch : 1
Updated theta: [[2.81179376]
 [1.04170895]
 [1.98749592]]
Previous Cost : 100 Next Cost : 1.004991409626457 Difference: 98.99500859037354
Previou

In [ ]:
#calculate the time taken to reach the convergence point 
#r=10000
start_time = time.time()
X1,Y1 = sample_generator(1000000)
data,t2 = SGD(X1,Y1,1000000,10000)
end_time = time.time()
print('Execution Time in seconds :',round(end_time - start_time,2) )

Streaming output truncated to the last 5000 lines.
 [1.00014794]
 [2.0005233 ]]
Previous Cost : 100 Next Cost : 0.998636740520577 Difference: 99.00136325947942
epoch : 167170
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167171
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167172
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167173
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167174
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167175
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167176
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167177
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167178
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
epoch : 167179
Updated theta: [[3.00004329]
 [1.00014794]
 [2.0005233 ]]
Previous Cost : 100 Next Cost : 0.998636740520577 Difference: 99.00136325947942
epoch : 167180
Updated theta: 

In [ ]:
#calculate the time taken to reach the convergence point 
#r=1000000
start_time = time.time()
X1,Y1 = sample_generator(1000000)
data,t2 = SGD(X1,Y1,1000000,1000000)
end_time = time.time()
print('Execution Time in seconds :',round(end_time - start_time,2) )

epoch : 0
Updated theta: [[0]
 [0]
 [0]]
Previous Cost : 100 Next Cost : 3.2455712315774505 Difference: 96.75442876842254
Previous Cost : 100 Next Cost : 1.480909130585813 Difference: 98.51909086941419
Previous Cost : 100 Next Cost : 1.2542441944469003 Difference: 98.7457558055531
Previous Cost : 100 Next Cost : 1.128869931805428 Difference: 98.87113006819457
Previous Cost : 100 Next Cost : 1.037856992309763 Difference: 98.96214300769023
Previous Cost : 100 Next Cost : 1.0077733106780267 Difference: 98.99222668932197
Previous Cost : 100 Next Cost : 0.9978860752257902 Difference: 99.0021139247742
Previous Cost : 100 Next Cost : 1.0173746454623742 Difference: 98.98262535453763
Previous Cost : 100 Next Cost : 1.0117136393807729 Difference: 98.98828636061923
Previous Cost : 100 Next Cost : 0.9989075346816153 Difference: 99.00109246531838
Previous Cost : 100 Next Cost : 0.9307218390378439 Difference: 99.06927816096216
Previous Cost : 100 Next Cost : 0.904005331319307 Difference: 99.09599466

In [ ]:
#checking out the dataset 
check= pd.read_csv("/content/drive/MyDrive/ML IITD/ass1/Q2/q2test.csv", header=None, skiprows=1)
check.head()

,0,1,2
0,16.678,13.018,45.537
1,6.583,-5.539,-1.170
2,-19.837,6.089,-3.646
3,-8.412,6.110,8.137
4,1.052,11.595,25.781


In [ ]:
# now I will compute the test error with respect to the prediction of the original hypothesis, and compare with the error obtained using the learned
## hypothesis in each case. 
# I will generate a function that will read the data from the dataset 
#I will split and initilise the Ytest and Xtest vectors 
def read_test():
    dataset= pd.read_csv("/content/drive/MyDrive/ML IITD/ass1/Q2/q2test.csv", header=None, skiprows=1)
    Ytest = dataset[2].values
    m = len(Ytest)
    Xtest = dataset[[0,1]]
    Xtest = np.array(Xtest)
    identity_arr = np.ones(m)
    identity_arr = identity_arr.reshape(m,1) #have to reshape into (m,1) vector form 
    Xtest= np.hstack((identity_arr,Xtest))
    return Xtest,Ytest

In [ ]:
# I will define a test function that will compute the test error 
def test_error():
    X,Y = read_test()
    original_theta = np.array([3,1,2])
    theta0t = np.array([2.94258289,1.04495778, 2.00450721])
    theta1t = np.array([2.99683404 , 1.00108993, 2.00074606])
    theta3t = np.array([2.96354437, 0.95692492, 1.98497986])
    theta_arr2=np.vstack((theta0t,theta1t,theta3t))  #will stack these values into a single 
    for i in range(0,3):
        original_cost = j_theta(X, Y, original_theta) #the cost computed from this dataset
        print('original cost :',original_cost) 
        my_cost = j_theta(X, Y, theta_arr2[i])  #the cost computed earleir, using the ttetas in the matrix 
        print('My Theta :',theta_arr2[i])  #the first value of array of thetas computed 
        print('My Cost :',my_cost) #the cost I calculated earlier 
        print('Error difference :',my_cost - original_cost) #the error difference 

In [ ]:
test_error()

original cost : 0.9829469215
My Theta : [2.94258289 1.04495778 2.00450721]
My Cost : 1.0819801635100443
Error difference : 0.09903324201004438
original cost : 0.9829469215
My Theta : [2.99683404 1.00108993 2.00074606]
My Cost : 0.9828868121965965
Error difference : -6.01093034034994e-05
original cost : 0.9829469215
My Theta : [2.96354437 0.95692492 1.98497986]
My Cost : 1.0943214807879795
Error difference : 0.11137455928797957
